In [1]:
import pandas as pd
import numpy as np

/var/folders/4k/4b01rf9j7cl6mcd3652m260w0000gn/T/ipykernel_38216/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_completa = pd.read_csv('../DATOS/Ford.csv')

In [3]:
#LIMPIEZA DATOS
data_completa=data_completa.drop(columns=["Date","Open","High","Low","Adj Close","Volume"])

In [4]:
#PREPARAR DATOS POR RANGOS
numero_periodos=5
# Función para determinar el estado según el cambio
def determinar_estado(cambio):
    if cambio > -0.4 and cambio < 0.4 :
        return 'Estable'
    elif cambio < 0:
        return 'Bajo'
    else:
        return 'Subio'
# Calcula la diferencia entre los valores de cierre consecutivos y asigna el resultado a la columna 'Cambio'
data_completa['Cambio'] = data_completa['Close'].diff()
# Aplica la función 'determinar_estado' a la columna 'Cambio' y crea una nueva columna 'EstadoNuevo' con los resultados
data_completa['EstadoNuevo'] = data_completa['Cambio'].apply(determinar_estado)



#CSV pruebas para excel 
data_completa['EstadoNuevo'].to_csv('../DATOS/Honda_1.csv')
# Divide el DataFrame en un array de DataFrames, cada uno representando un periodo
data_por_periodo = np.array_split(data_completa,numero_periodos)




/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
frecuencias_observadas = []

for i in range(numero_periodos):
    # Calcula la tabla de frecuencias observadas usando crosstab entre el estado actual y el estado anterior (shift)
    frecuencia_obs = pd.crosstab(data_por_periodo[i]['EstadoNuevo'].shift(), data_por_periodo[i]['EstadoNuevo'], normalize=False)
    print("FRECUENCIA OBSERVADA - PERIODO", i+1, "\n", frecuencia_obs, "\n")
    # Agrega la frecuencia observada a la lista
    frecuencias_observadas.append(frecuencia_obs)


FRECUENCIA OBSERVADA - PERIODO 1 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            0        5      0
Estable         4       34      4
Subio           1        4      0 

FRECUENCIA OBSERVADA - PERIODO 2 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            3        3      3
Estable         2       23      6
Subio           4        5      2 

FRECUENCIA OBSERVADA - PERIODO 3 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            6        4      4
Estable         4       11      4
Subio           5        3     10 

FRECUENCIA OBSERVADA - PERIODO 4 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            7        5      6
Estable         9        5      4
Subio           3        8      4 

FRECUENCIA OBSERVADA - PERIODO 5 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            2        5      4
Estable         6       18      5
S

In [6]:
porcentaje_observado = [None] * numero_periodos

for i in range(0, numero_periodos):
    # Calcula los porcentajes observados dividiendo las frecuencias observadas por la suma total de cada periodo
    porcentaje_observado[i] = frecuencias_observadas[i] / frecuencias_observadas[i].sum()
    #Transpone la matriz para que sea más fácil de leer y comparar con el Excel
    porcentaje_observado[i] = porcentaje_observado[i].T
    
print("Porcentaje relativo:\n", porcentaje_observado[1]) #Ejemplo de impresion de un periodo


Porcentaje relativo:
 EstadoNuevo      Bajo   Estable     Subio
EstadoNuevo                              
Bajo         0.333333  0.222222  0.444444
Estable      0.096774  0.741935  0.161290
Subio        0.272727  0.545455  0.181818


In [7]:
# Calcula la tabla de frecuencias esperadas usando crosstab entre el estado actual y el estado anterior (shift) en todo el conjunto de datos
frecuencia_esperada= pd.crosstab(data_completa['EstadoNuevo'].shift(), data_completa['EstadoNuevo'])
print("PROBABILIDAD OBSERVADA \n",frecuencia_esperada,"\n")


PROBABILIDAD OBSERVADA 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo           18       23     18
Estable        25       93     23
Subio          16       25     19 



In [8]:
# Calcula los porcentajes esperados dividiendo las frecuencias esperadas por la suma total de todas las frecuencias esperadas
porcentaje_esperado = frecuencia_esperada / frecuencia_esperada.sum()
#Transpone la matriz para que sea más fácil de leer y comparar con el Excel
porcentaje_esperado = porcentaje_esperado.T
print(porcentaje_esperado)


EstadoNuevo      Bajo   Estable     Subio
EstadoNuevo                              
Bajo         0.305085  0.423729  0.271186
Estable      0.163121  0.659574  0.177305
Subio        0.300000  0.383333  0.316667


In [9]:
ChiObs = 0
for i in range(0, numero_periodos): 
     # Suma acumulativa de los valores chi-cuadrado observados se aplica la formula vista en clase
    ChiObs+= np.sum(frecuencias_observadas[i].sum().values*(porcentaje_observado[i].values-porcentaje_esperado.values)**2/porcentaje_esperado.values)
    print(frecuencias_observadas[i].sum().values)
    print(porcentaje_observado[i].values)
    print(porcentaje_esperado.values)
    break
print(f'Chi Observado: {ChiObs}')


[ 5 43  4]
[[0.         0.8        0.2       ]
 [0.11627907 0.79069767 0.09302326]
 [0.         1.         0.        ]]
[[0.30508475 0.42372881 0.27118644]
 [0.16312057 0.65957447 0.17730496]
 [0.3        0.38333333 0.31666667]]
Chi Observado: 62.74002102074414


In [10]:
from scipy.stats import chi2
grados_libertad_total = 0
sum_filas = 0
# Iteración a través de las frecuencias observadas
for df in frecuencias_observadas:
    num_filas, num_columnas = df.shape
    # Suma el número de filas al total para los grados de libertad
    sum_filas += num_filas
# Calcula los grados de libertad
grados_libertad = (sum_filas - 1) * (num_columnas - 1)

print("Grados de libertad total:", grados_libertad)
# Calcula el valor chi-cuadrado teórico para un nivel de significancia del 5% y los grados de libertad calculados
chiTeorico = chi2.ppf(0.95, grados_libertad)
print("Chi Teorico= ",chiTeorico)

Grados de libertad total: 28
Chi Teorico=  41.33713815142739


In [11]:
# Compara el valor chi-cuadrado observado con el valor chi-cuadrado teórico crítico
if(ChiObs < chiTeorico):
    print("Se acepta la hipótesis nula, por lo tanto se considera estacionario en el tiempo.")
else:
    print("Se rechaza la hipótesis nula, por lo tanto no se considera estacionario en el tiempo.")

Se rechaza la hipótesis nula, por lo tanto no se considera estacionario en el tiempo.
